In [1]:
import re


class Pipeline:
    s1 = 'https://api.demographicspro.com/10/api/get_aggregate_analysis?for=followers_of_handle&data='    
    s2 = '&network=instagram'
    def __init__(self, namelistPath, downloadFilePath):
        self.namelistPath = namelistPath
        self.downloadFilePath = downloadFilePath
        

## 完成任务先

In [9]:
#get the name list 
import re

namelistPath = "/Users/zhangyuhao/Downloads/influencers_active.csv"
lst = []
with open(namelistPath, 'r') as f:
    lines = f.readlines()
    for line in lines:
        m = re.match('(.*)\n$', line)
        lst.append(m.group(1))

In [2]:
lst = ['endorsify','mayahirasedo','yungwatergun']

In [11]:
#pull data from Demopro using the namelist
#url stores all api url to pull data from Demopro

s1 = 'https://api.demographicspro.com/10/api/get_aggregate_analysis?for=followers_of_handle&data='    
s2 = '&network=instagram'

url = []
for name in lst[0:]:
    url.append(s1+name+s2)

In [12]:
url

['https://api.demographicspro.com/10/api/get_aggregate_analysis?for=followers_of_handle&data=endorsify&network=instagram',
 'https://api.demographicspro.com/10/api/get_aggregate_analysis?for=followers_of_handle&data=mayahirasedo&network=instagram',
 'https://api.demographicspro.com/10/api/get_aggregate_analysis?for=followers_of_handle&data=yungwatergun&network=instagram']

In [4]:
#This is the function to process the data from Demopro

"""
code_path: str #the code of Demopro's taxonomy
"""

class ePreprocess(object):
    
    def __init__(self, code_path):
        self.codePath = code_path
        self.sections = ['Gender', 'Age - Census', 'Age - Nielsen', 'Location: by country', 'Location: by city', 'Personal income', 'Likes & interests', 'Brands']
        self.code_dict = self.loadCode(self.codePath)
        
        
        
    def loadCode(self, codePath):
        try:
        #load code
            with open(codePath, 'r') as f:
                code_dict = json.load(f)
            print("Code loading success")
            return code_dict
        except:
            print("Code loading failed")
            
            
            
    def process(self, data):
    
        #process every section
        try:
            for section in data['analysis']['analysis_data']:
                section_name = section['section_name']

                #decode the taxonomy
                for row in section['section_rows']:
                    code = row['code']
                    row['code'] = self.code_dict[section_name][code]
            print("Decoding finished")
        except:
            print("Decoding failed")
            
        return data

In [13]:
# Now we can pull data from demopro using these url by requests
# And save them as json format locally

from urllib import request

import requests
import json
import re
import time

process = ePreprocess(code_path = '/Users/zhangyuhao/spiders/endorsify/codes/codes.json')

errorList = []

for i in range(len(lst)):
    u = url[i]
    name = re.match('.*data=(.*)&network',u).group(1)
    
    data = requests.get(u, auth=('endorsify_api','v46yitpl95hf'))
    time.sleep(5)
    try:
        data = json.loads(data.text)
    except:
        errorList.append((name, 504))
        continue
    
    
    #if error occures　
    
    if data['status'] == "Error":
        print("Analysis of {} failed: {}".format(name, data['error_details']))
        errorList.append((name, data['error_details']))
        pass
        
    while data['status'] =="Queued":
        data = requests.get(u, auth=('endorsify_api','v46yitpl95hf'))
        time.sleep(5)
        data = json.loads(data.text)
        print("analysis of {} queued, waiting".format(name))
        
    data = process.process(data)
    
    with open('/Users/zhangyuhao/spiders/endorsify/influencers/Round2/temp/{}.json'.format(name), 'w') as f:
        json.dump(data, f)
        print('Decoding finished: {}    ({}/{})'.format(name, i, 900))

with open("/Users/zhangyuhao/spiders/endorsify/influencers/Round2/errors.json", 'a') as f:
    json.dump(errorList, f)
    print("Error info recorded")

Code loading success
analysis of endorsify queued, waiting
analysis of endorsify queued, waiting
analysis of endorsify queued, waiting
analysis of endorsify queued, waiting
analysis of endorsify queued, waiting
analysis of endorsify queued, waiting
analysis of endorsify queued, waiting
analysis of endorsify queued, waiting
analysis of endorsify queued, waiting
Decoding finished
Decoding finished: endorsify    (0/900)
Decoding finished
Decoding finished: mayahirasedo    (1/900)
Decoding finished
Decoding finished: yungwatergun    (2/900)
Error info recorded


In [8]:
errorList

[]

In [58]:
with open("/Users/zhangyuhao/spiders/endorsify/influencers/Round2/errors.json", 'a') as f:
    for error in errorList:
        json.dump(error, f)
    print("Error info recorded")

Error info recorded


In [60]:
data.text

"<html><body><h1>504 Gateway Time-out</h1>\nThe server didn't respond in time.\n</body></html>\n"

In [36]:
data.text

'{"status": "Complete", "percent_complete": 100, "error_details": "", "analysis": {"analysis_name": "Followers of @the_beauty_issue", "number_processed": 8540, "analysis_id": 1061080, "number_submitted": 20102, "analysis_data": [{"section_rows": [{"estimated_size": 4065, "code": "GEb", "num": 1579, "pct": 20.21767}, {"estimated_size": 16037, "code": "GEn", "num": 6231, "pct": 79.78233}], "section_total": 7810, "section_name": "Gender"}, {"section_rows": [{"estimated_size": 81, "code": "ACb", "num": 5, "pct": 0.402901}, {"estimated_size": 794, "code": "ACn", "num": 49, "pct": 3.948429}, {"estimated_size": 7062, "code": "ACd", "num": 436, "pct": 35.132957}, {"estimated_size": 7289, "code": "ACr", "num": 450, "pct": 36.26108}, {"estimated_size": 2819, "code": "ACf", "num": 174, "pct": 14.020951}, {"estimated_size": 1247, "code": "ACg", "num": 77, "pct": 6.204674}, {"estimated_size": 697, "code": "AC8", "num": 43, "pct": 3.464948}, {"estimated_size": 65, "code": "ACe", "num": 4, "pct": 0.3

# Now upload data to the cloud

In [2]:
#get list of influencer data file names

import os
path = '/Users/zhangyuhao/spiders/endorsify/influencers/Round2'
files = os.listdir(path)
print(files)

['annabaun.json', 'ichmiles.json', 'lebarboteur.json', 'lyesk.json', 'sugsean.json', 'svistal13.json', 'urbantraveller.json', 'tommiiib.json', 'zacwhiteee.json', 'ellabrooksblog.json', 'bamitsjoanne.json', 'chiaradalben.json', 'stefanopirini.json', 'mentaylunares.json', 'fawad.aly.json', 'carolinegleich.json', 'gofitjo.json', 'closetfatgirl.json', 'nanukboy.json', 'elliottsailors.json', 'tobilikee.json', 'kerimayar.json', 'makeupbecca.json', 'oliamajd.json', 'myviewinheels.json', 'spizoiky.json', 'melissackoh.json', 'maine_blonde.json', 'lolitamas.json', 'leandre.renoir.json', 'edwardjayhan.json', '_lagiuditta.json', 'hasna_lifestyle.json', 'r4s.json', 'kymgrimshaw.json', 'jordanmariebrennan.json', 'gerahii.json', 'kaseystjohn.json', 'missnikkileigh.json', 'alixelay.json', 'cookingvideos.json', 'emily_soule.json', 'ellenvlora.json', 'thebeautybeau.json', 'atlasandmason.json', 'anisasojka.json', 'jirkavinse.json', 'dennisvanpeel.json', 'hellodarlingblog.json', 'maggieeats.json', 'tayste

In [22]:
#remove "errors.json" from fileNamelist
files.remove('errors.json')

#remove files which is not .json
def clean(files):
    for name in files:
        if re.match('.*.json$', name):
            pass
        else:
            files.remove(name)
            
    return files
            
files = clean(files)

In [6]:
# define data pipeline
def dataPipeline(data):
    output = dict()
    
    analysis = data['analysis']
    output['analysisName'] = re.match('.*@(.*)',analysis['analysis_name']).group(1)
    output['number_followers'] = analysis['number_submitted']
    output['network'] = analysis['network']
    output['analysis_date'] = analysis['analysis_date']
    
    #load detail analysis 
    detail = analysis['analysis_data']

    for section in detail:
        section_name = section['section_name']
        output[section_name] = section['section_rows']
    return output

In [20]:
# connect to AWS RDS

import MySQLdb

db = MySQLdb.connect("endorsifydb.czczswhwmw1x.us-east-2.rds.amazonaws.com", "admin", 'endorsify', 'influencerDB')  #connection

cursor = db.cursor()  #piont
      

In [23]:
import json
import re

totalNumOfFiles = len(files)
curProcess = 1

for name in files:
    with open('/Users/zhangyuhao/spiders/endorsify/influencers/Round2/{}'.format(name), 'r') as f:
        data_raw = json.load(f)
        output = dataPipeline(data_raw)
        
        #Insert data into database
        try:
            sql = "INSERT INTO primaryDB(analysisName, number_followers, network, analysis_date,\
                    Gender, AgeCensus, AgeNielsen, LocationByCountry, LocationByCity, PersonalIncome, LikesInterests, Brands)\
                    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            values = (output['analysisName'], output['number_followers'], output['network'], \
                      output['analysis_date'],json.dumps(output['Gender']), json.dumps(output['Age - Census']),\
                      json.dumps(output['Age - Nielsen']), json.dumps(output['Location: by country']),\
                      json.dumps(output['Location: by city']), json.dumps(output['Personal income']),\
                      json.dumps(output['Likes & interests']), json.dumps(output['Brands']))
            cursor.execute(sql,values)
            print('Insert data of {} success!   {}/{}'.format(output['analysisName'], curProcess, totalNumOfFiles))
            curProcess+=1

        except:
            rollback()
            print("Fail to insert Data")
                    
db.commit()
print('--------------Upload Finished!------------------')

Insert data of lyesk success!   1/882
Insert data of sugsean success!   2/882
Insert data of svistal13 success!   3/882
Insert data of urbantraveller success!   4/882
Insert data of tommiiib success!   5/882
Insert data of zacwhiteee success!   6/882
Insert data of ellabrooksblog success!   7/882
Insert data of bamitsjoanne success!   8/882
Insert data of chiaradalben success!   9/882
Insert data of stefanopirini success!   10/882
Insert data of mentaylunares success!   11/882
Insert data of fawad.aly success!   12/882
Insert data of carolinegleich success!   13/882
Insert data of gofitjo success!   14/882
Insert data of closetfatgirl success!   15/882
Insert data of nanukboy success!   16/882
Insert data of elliottsailors success!   17/882
Insert data of tobilikee success!   18/882
Insert data of kerimayar success!   19/882
Insert data of makeupbecca success!   20/882
Insert data of oliamajd success!   21/882
Insert data of myviewinheels success!   22/882
Insert data of spizoiky succe

Insert data of charlottelum success!   179/882
Insert data of momdutyblog success!   180/882
Insert data of queencarlene success!   181/882
Insert data of gvozdishe success!   182/882
Insert data of meganadelaide success!   183/882
Insert data of marymisantropic success!   184/882
Insert data of beccatapert success!   185/882
Insert data of toks_amy success!   186/882
Insert data of suzannespiegoski success!   187/882
Insert data of hx1125 success!   188/882
Insert data of josepalbert success!   189/882
Insert data of thesalty.dreamers success!   190/882
Insert data of withorwithoutshoes success!   191/882
Insert data of hieucow success!   192/882
Insert data of beingzhenya success!   193/882
Insert data of luda_ success!   194/882
Insert data of yoga_ky success!   195/882
Insert data of a_ontheroad success!   196/882
Insert data of dshukin success!   197/882
Insert data of brandonma1212 success!   198/882
Insert data of mattcherub success!   199/882
Insert data of cuordicarciofo succe

Insert data of thomaspoulet success!   355/882
Insert data of jessakae success!   356/882
Insert data of robintuck1 success!   357/882
Insert data of sherylnsy success!   358/882
Insert data of themoptop success!   359/882
Insert data of theldndiaries success!   360/882
Insert data of livinglesh success!   361/882
Insert data of sir_ndrewsilver success!   362/882
Insert data of hanaramadan__ success!   363/882
Insert data of vivid success!   364/882
Insert data of posh_scene success!   365/882
Insert data of kristywicks success!   366/882
Insert data of girlxdeparture success!   367/882
Insert data of girltravelboss success!   368/882
Insert data of brioenfurecida success!   369/882
Insert data of hello.drifter success!   370/882
Insert data of minimal_duck success!   371/882
Insert data of ariannahicks success!   372/882
Insert data of forkmeetsfood success!   373/882
Insert data of theblacklayers success!   374/882
Insert data of cocoscurvycloset success!   375/882
Insert data of vic

Insert data of jared_ldn success!   530/882
Insert data of brandonstansell success!   531/882
Insert data of inn8chiro success!   532/882
Insert data of joao.bernardino success!   533/882
Insert data of danielo_costa success!   534/882
Insert data of dillonpena success!   535/882
Insert data of mandymartini success!   536/882
Insert data of isabellemusic success!   537/882
Insert data of jamieout success!   538/882
Insert data of be_parisien success!   539/882
Insert data of cristinaneros success!   540/882
Insert data of xxjmitch success!   541/882
Insert data of danilocarnevale success!   542/882
Insert data of polydeux success!   543/882
Insert data of vladislavkarpyuk success!   544/882
Insert data of the_foodiediaries success!   545/882
Insert data of joellenlove success!   546/882
Insert data of sieklas success!   547/882
Insert data of courtneyveronica success!   548/882
Insert data of jrthewitness success!   549/882
Insert data of martinsalomonjr success!   550/882
Insert data 

Insert data of cannellevanille success!   707/882
Insert data of london_ellis success!   708/882
Insert data of elizabethluiss success!   709/882
Insert data of juanjerez success!   710/882
Insert data of dandyinthebronx success!   711/882
Insert data of samearp success!   712/882
Insert data of gingerfitspo success!   713/882
Insert data of antoniaortega success!   714/882
Insert data of elle_kae success!   715/882
Insert data of joycorrigan success!   716/882
Insert data of missalissa success!   717/882
Insert data of thefashionmuse_ success!   718/882
Insert data of iamfasholica success!   719/882
Insert data of paperboyo success!   720/882
Insert data of ashwisdom success!   721/882
Insert data of mossonyi success!   722/882
Insert data of mchllengo success!   723/882
Insert data of philbottenberg success!   724/882
Insert data of hellorigby success!   725/882
Insert data of wandergasm success!   726/882
Insert data of _epicspence success!   727/882
Insert data of robinsebastians s

In [18]:
files.index('.ipynb_checkpoints')

667

In [19]:
cursor.close()  #close pointer
db.close()  

# convert data to readable ones

In [1]:
#get list of influencer data file names

import os
path = '/Users/zhangyuhao/spiders/endorsify/influencers/Round2/temp'
files = os.listdir(path)
print(files)

['mayahirasedo.json', 'endorsify.json', 'yungwatergun.json']


In [44]:
import json
for name in files:
    with open('/Users/zhangyuhao/spiders/endorsify/influencers/Round2/temp/{}'.format(name), 'r') as f:
        data_raw = json.load(f)
        data = ConvertToHumanRead(data_raw)
        
        with open('/Users/zhangyuhao/spiders/endorsify/influencers/Round2/temp/{}'.format('Processed'+name), 'a') as file:
            json.dump(data, file)

{'status': 'Complete',
 'percent_complete': 100,
 'error_details': '',
 'analysis': {'analysis_name': 'Followers of @mayahirasedo',
  'number_processed': 7642,
  'analysis_id': 644915,
  'number_submitted': 25074,
  'analysis_data': [{'section_rows': [{'estimated_size': 9450,
      'code': 'Male',
      'num': 2313,
      'pct': 37.689425},
     {'estimated_size': 15624,
      'code': 'Female',
      'num': 3824,
      'pct': 62.310575}],
    'section_total': 6137,
    'section_name': 'Gender'},
   {'section_rows': [{'estimated_size': 30,
      'code': 'Age 16 and under',
      'num': 3,
      'pct': 0.117739},
     {'estimated_size': 1850,
      'code': 'Age 17 to 19',
      'num': 188,
      'pct': 7.378336},
     {'estimated_size': 15952,
      'code': 'Age 20 to 24',
      'num': 1621,
      'pct': 63.618524},
     {'estimated_size': 4901,
      'code': 'Age 25 to 29',
      'num': 498,
      'pct': 19.544741},
     {'estimated_size': 1545,
      'code': 'Age 30 to 34',
      'num'

In [43]:
def ConvertToHumanRead(data_raw):
    import heapq

    analysis = data_raw['analysis']
    analysis_data = analysis['analysis_data']

    analysisDic = dict()
    analysisDic['analysis_name'] = analysis['analysis_name']
    analysisDic['number_followers'] = analysis['number_submitted']

    for i in range(len(analysis_data)):
        item = analysis_data[i]

        #name of this part of analysis, ie, Gender
        itemName = item['section_name']

        #rows is the analysis content 
        rows = item['section_rows']

        #analysisDict[itemName][]
        sectionDic = dict()
        if itemName == 'Gender':
            for section in rows:
                sectionDic[section['code']] = dict()
                sectionDic[section['code']]['num']=section['num']
                sectionDic[section['code']]['pct']=section['pct']

        #rows stores dicts, in each dict, there is a section of the analysis, ie, 
        else:
            heapSize = 3
            #filter, only keep top3 section, which takes the largest part
            heap = []
            for i in range(len(rows)):
                if len(heap)<3:
                    heapq.heappush(heap, (rows[i]['pct'], i))
                else:
                    if rows[i]['pct']>heap[0][1]:
                        heapq.heappushpop(heap, (rows[i]['pct'], i))

            heap.sort(key = lambda x: x[0], reverse = True)
            for i in range(len(heap)):
                heap[i] = rows[heap[i][1]]

            for section in heap:
                sectionDic[section['code']] = dict()
                sectionDic[section['code']]['num']=section['num']
                sectionDic[section['code']]['pct']=section['pct']

        analysisDic[itemName] = sectionDic

    return analysisDic

In [41]:
analysisDic

{'analysis_name': 'Followers of @mayahirasedo',
 'number_followers': 25074,
 'Gender': {'Male': {'num': 2313, 'pct': 37.689425},
  'Female': {'num': 3824, 'pct': 62.310575}},
 'Age - Census': {'Age 20 to 24': {'num': 1621, 'pct': 63.618524},
  'Age 25 to 29': {'num': 498, 'pct': 19.544741},
  'Age 17 to 19': {'num': 188, 'pct': 7.378336}},
 'Age - Nielsen': {'Age 21 to 24': {'num': 1045, 'pct': 41.012559},
  'Age 18 to 20': {'num': 760, 'pct': 29.827316},
  'Age 25 to 29': {'num': 498, 'pct': 19.544741}},
 'Location: by country': {'Australia': {'num': 2339, 'pct': 33.649835},
  'United States': {'num': 2142, 'pct': 30.81571},
  'Canada': {'num': 179, 'pct': 2.575169}},
 'Location: by city': {'Sydney': {'num': 412, 'pct': 6.336258},
  'Perth': {'num': 113, 'pct': 1.737857},
  'Stuttgart': {'num': 5, 'pct': 0.09873}},
 'Personal income': {'$20,000 - $29,999': {'num': 566, 'pct': 19.840491},
  '$10,000 - $19,999': {'num': 559, 'pct': 19.595114},
  '$30,000 - $39,999': {'num': 410, 'pct': 